# Valutazione Funzione di Rilevamento

In [74]:
import pandas as pd

dati=pd.read_csv('risultati.csv' ,usecols=["Id","Prima etichetta","Pr Prima etichetta","Luogo","Orario","Verita Seconda etichetta","Tipo situazione","Seconda etichetta", "Pr Seconda etichetta","Incertezza Sistema"])
dati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        599 non-null    int64  
 1   Prima etichetta           599 non-null    object 
 2   Pr Prima etichetta        599 non-null    int64  
 3   Luogo                     599 non-null    object 
 4   Orario                    599 non-null    int64  
 5   Verita Seconda etichetta  599 non-null    object 
 6   Tipo situazione           599 non-null    object 
 7   Seconda etichetta         599 non-null    object 
 8   Pr Seconda etichetta      599 non-null    float64
 9   Incertezza Sistema        599 non-null    float64
dtypes: float64(2), int64(3), object(5)
memory usage: 46.9+ KB


### Positivi=Sospetti ,    Negativi=Non sospetti

## Actual condition

In [75]:
N=dati[dati['Verita Seconda etichetta']=="Non sospetto"]['Id'].count() #numero casi negativi reali nei dati
P=dati[dati['Verita Seconda etichetta']=="Sospetto"]['Id'].count() #numero casi positivi reali nei dati
T=N+P #casi totali
print(N,P,T)

527 72 599


## Predicted condition

In [93]:
esatti=dati[dati['Verita Seconda etichetta']==dati['Seconda etichetta']]# dati per cui la predizione è esatta
TT=esatti['Id'].count() #numero casi per cui la predizione è esatta
TN=esatti[esatti['Seconda etichetta']=="Non sospetto"]['Id'].count() # numero casi  correttamente etichettati come negativo
TP=esatti[esatti['Seconda etichetta']=="Sospetto"]['Id'].count() # numero casi correttamente etichettati come positivi
print(TN,TP,TT)


527 24 551


In [95]:
errati=dati[dati['Verita Seconda etichetta']!=dati['Seconda etichetta']]# dati per cui la predizione è errata
TE=errati['Id'].count() #numero casi per cui la predizione è errata
FN=errati[errati['Seconda etichetta']=="Non sospetto"]['Id'].count() # numero casi erroneamente etichettati come negativi
FP=errati[errati['Seconda etichetta']=="Sospetto"]['Id'].count() # numero casi erroneamente etichettati come positivi
print(FN,FP,TE)

48 0 48


## Metrics

In [97]:
Prevalence=P/(P+N) #prevalenza
Prevalence

0.12020033388981637

### Independent on the prevalence

### Dependent on the prevalence